In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.H2('Sogoba was here')),
    html.Hr(),
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'),
    html.Div(
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain Rescue', 'value': 'mountain'},
            {'label': 'Disaster Rescue', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'},
        ],
        value='Reset',
        labelStyle={'display': 'inline-block'}
    )
    #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    ),
    html.Hr(),
    
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action ="native",
        sort_action ="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,    
    ),
    html.Br(),
    html.Hr(),
    # create a div for the map data
    html.Div(className='row',
         style={'display' : 'flex'},
         children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',

                ),
            html.Div(
                id='map-id',
                className='col s12 m6',
                )
        ]),
])

#############################################
# Interaction Between Components / Controller
#############################################
                               
    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        if (filter_type == 'all'):
            df = pd.DataFrame.from_records(shelter.read({}))
        elif(filter_type == 'disaster'):
            df =pd.DataFrame.from_records(shelter.read({"breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gt":"20"},"age_upon_outcome_in_weeks":{"$lt":"300"}}))
        elif(filter_type == 'water'):
            df =pd.DataFrame.from_records(shelter.read({"breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]}, "sex_upon_outcome":"Intact Female", "age_upon_outcome_in_weeks":{"$gt":"26"},"age_upon_outcome_in_weeks":{"$lt":"156"}}))
        elif(filter_type == 'mountain'):
            df =pd.DataFrame.from_records(shelter.read({"breed":{"$in":["German Shepherd,","Alaskan Malamute","Old English Sheepdog", "Siberian Husky", "Rottweiler" ]}, "sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gt":"26"},"age_upon_outcome_in_weeks":{"$lt":"156"}}))
        else:
            df = pd.DataFrame.from_records(shelter.read({}))
            
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records') 
        return (data,columns)



@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed')
    return [
        dcc.Graph(            
            figure = fig
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    if viewData is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    loc = [dff.location_lat[0], dff.location_long[0]]
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=loc, zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=loc, children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1('Animal Name'),
                    html.P(dff.name[0])
                ])
            ])
        ])
    ]




app